In [2]:
import pandas as pd
import urllib.parse

def fetch_data(url):
    print("Fetching data from API...")
    try:
        df = pd.read_json(url)
        print(f"Data successfully retrieved! Rows: {df.shape[0]}")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

api_url1 = (
    "https://data.cityofchicago.org/resource/ggws-77ih.json?"
    "$query=SELECT%20measurement_title,%20measurement_description,%20measurement_type,%20"
    "measurement_medium,%20measurement_time,%20measurement_value,%20units,%20"
    "units_abbreviation,%20measurement_period_type,%20data_stream_id,%20resource_id,%20"
    "measurement_id,%20record_id,%20latitude,%20longitude,%20location%20"
    "WHERE%20measurement_time%20BETWEEN%20%272017-01-01T00:00:00%27::floating_timestamp%20"
    "AND%20%272017-03-31T23:59:59%27::floating_timestamp%20"
    "ORDER%20BY%20measurement_time%20DESC%20NULL%20FIRST,%20data_stream_id%20ASC%20NULL%20LAST%20"
    "LIMIT%201000000"
)

raw_query = """
SELECT measurement_title, measurement_description, measurement_type, measurement_medium,
       measurement_time, measurement_value, units, units_abbreviation,
       measurement_period_type, data_stream_id, resource_id, measurement_id, record_id,
       latitude, longitude, location
WHERE measurement_time BETWEEN '2017-01-01T00:00:00'::floating_timestamp
  AND '2017-06-30T23:45:00'::floating_timestamp
  AND caseless_contains(units, 'degrees Celsius')
  AND caseless_contains(measurement_type, 'Temperature')
  AND caseless_contains(measurement_medium, 'atmosphere')
ORDER BY data_stream_id ASC NULL LAST
LIMIT 400000
"""

encoded_query = urllib.parse.quote(raw_query, safe='')

api_url2 = f"https://data.cityofchicago.org/resource/ggws-77ih.json?$query={encoded_query}"

df1 = fetch_data(api_url1)
df2 = fetch_data(api_url2)

if df1 is not None and df2 is not None:
    df_combined = pd.concat([df1, df2], ignore_index=True)
else:
    df_combined = df1 if df1 is not None else df2

df = df_combined.copy()

print("First 5 rows of the combined dataset:")
print(df.head())


Fetching data from API...
Data successfully retrieved! Rows: 605620
Fetching data from API...
Data successfully retrieved! Rows: 360895
First 5 rows of the combined dataset:
                               measurement_title         measurement_type  \
0     Langley - Cumulus: Bioswale Calibrated VWC             SoilMoisture   
1    Langley - Cumulus: Weather Station Rainfall  CumulativePrecipitation   
2    Langley - Cumulus: Weather Station Air Temp              Temperature   
3    Langley - Cumulus: Weather Station Pressure     DifferentialPressure   
4  Langley - Cumulus: Weather Station Wind Speed                WindSpeed   

  measurement_medium    measurement_time  measurement_value  \
0                 NA 2017-03-31 23:59:56               32.0   
1                 NA 2017-03-31 23:59:56                0.0   
2                 NA 2017-03-31 23:59:56                0.0   
3                 NA 2017-03-31 23:59:56                0.0   
4                 NA 2017-03-31 23:59:56        

In [5]:
import altair as alt
import pandas as pd

print(df.head())
print(df.shape)

df_sample = df.sample(n=5000)

df['measurement_time'] = pd.to_datetime(df['measurement_time']) 
df['date'] = df['measurement_time'].dt.strftime('%Y-%m-%d')

df_agg = df.groupby('date')['measurement_value'].mean().reset_index(name='avg_temp')



                               measurement_title         measurement_type  \
0     Langley - Cumulus: Bioswale Calibrated VWC             SoilMoisture   
1    Langley - Cumulus: Weather Station Rainfall  CumulativePrecipitation   
2    Langley - Cumulus: Weather Station Air Temp              Temperature   
3    Langley - Cumulus: Weather Station Pressure     DifferentialPressure   
4  Langley - Cumulus: Weather Station Wind Speed                WindSpeed   

  measurement_medium    measurement_time  measurement_value  \
0                 NA 2017-03-31 23:59:56               32.0   
1                 NA 2017-03-31 23:59:56                0.0   
2                 NA 2017-03-31 23:59:56                0.0   
3                 NA 2017-03-31 23:59:56                0.0   
4                 NA 2017-03-31 23:59:56                0.0   

                              units units_abbreviation  \
0  Percent Volumetric Water Content             %(vwc)   
1                            inches       

In [3]:

df_agg['date'] = pd.to_datetime(df_agg['date'])
df_agg['day_of_week'] = df_agg['date'].dt.dayofweek  # Monday=0, Sunday=6
df_agg['week_of_year'] = df_agg['date'].dt.isocalendar().week


calendar_heatmap = alt.Chart(df_agg).mark_rect().encode(
    x=alt.X('day_of_week:O', title="Day of Week",
            axis=alt.Axis(
                labelExpr="datum.value == 0 ? 'Mon' : datum.value == 1 ? 'Tue' : datum.value == 2 ? 'Wed' : datum.value == 3 ? 'Thu' : datum.value == 4 ? 'Fri' : datum.value == 5 ? 'Sat' : 'Sun'"
            )),
    y=alt.Y('week_of_year:O', title="Week of Year"),
    color=alt.Color('avg_temp:Q', title="Avg Temp (°C)", scale=alt.Scale(scheme='reds')),
    tooltip=[
        alt.Tooltip('date:T', title='Date'),
        alt.Tooltip('avg_temp:Q', title='Avg Temp (°C)', format='.1f')
    ]
).properties(
    width=500,
    height=300,
    title="Calendar Heatmap: Daily Average Temperature"
)

calendar_heatmap


alt.Chart(...)

In [6]:
df_temp = df[df['measurement_type'].str.contains('Temperature', case=False, na=False)].copy()


# Filter 
df_temp = df_temp[(df_temp['measurement_value'] > -40) & (df_temp['measurement_value'] < 45)]
df_temp = df_temp.sample(n=5000)

# Create a 2D-binned heatmap:
spatial_heatmap = alt.Chart(df_temp).mark_rect().encode(
    x=alt.X('longitude:Q',
            bin=alt.Bin(maxbins=40),
            title='Longitude'),
    y=alt.Y('latitude:Q',
            bin=alt.Bin(maxbins=40),
            title='Latitude'),
    color=alt.Color('mean(measurement_value):Q',
                    title='Avg Temperature (°C)',
                    scale=alt.Scale(scheme='reds')),
    tooltip=[
        alt.Tooltip('mean(measurement_value):Q', title='Avg Temp (°C)', format='.1f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=500,
    height=400,
    title='Spatial Heatmap: Average Temperature Distribution'
)

spatial_heatmap



/opt/anaconda3/envs/cs418env/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/cs418env/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/envs/cs418env/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/e

alt.Chart(...)

In [17]:
import altair as alt
import pandas as pd



selected_types = ['Temperature', 'RelativeHumidity', 'CumulativePrecipitation', 'SoilMoisture', 'WindSpeed']

# filter & aggregate into agg_data
df_filtered = df[df['measurement_type'].isin(selected_types)].copy()
df_filtered['measurement_time'] = pd.to_datetime(df_filtered['measurement_time'])
df_filtered['date'] = df_filtered['measurement_time'].dt.normalize()

agg_data = (
    df_filtered
      .groupby(['date','measurement_type'])['measurement_value']
      .mean()
      .reset_index(name='avg_value')
)



agg_data = agg_data[
    (agg_data['date'] >= '2017-03-13') &
    (agg_data['date'] <= '2017-03-22')
].copy()
# days since epoch
agg_data['date_day'] = (
    (agg_data['date'] - pd.Timestamp('2017-03-13'))
    // pd.Timedelta('1d')
).astype(int)

min_day = agg_data['date_day'].min()
max_day = agg_data['date_day'].max()


day_slider = alt.binding_range(
    name='Show up to: ',
    min=min_day,
    max=max_day,
    step=1          
)
end_day = alt.param(
    'end_day', 
    bind=day_slider,
    value=max_day  
)


base = (
    alt.Chart(agg_data)
       .add_params(end_day)
       .transform_filter('datum.date_day <= end_day')
       .mark_line(point=True)
       .encode(
           x=alt.X('date:T', title='Date'),
           y=alt.Y('avg_value:Q', title='Daily Avg Value'),
           color=alt.Color('measurement_type:N', legend=None)
       )
       .properties(width=250, height=150)
)

# — 4) Facet into small multiples —
faceted_chart = base.facet(
    facet=alt.Facet('measurement_type:N', title='Measurement Type'),
    columns=2
).properties(
    title='Daily Averages by Type (Mar 1–22, 2017)'
).resolve_scale(y='independent')

faceted_chart



import json, os


with open('specs/faceted_slider.json', 'w') as f:
    json.dump(faceted_chart.to_dict(), f, indent=2)
print("Wrote specs/faceted_slider.json")




Wrote specs/faceted_slider.json


/opt/anaconda3/envs/cs418env/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [ ]:
# print the entire measurement_time column
print(df['measurement_time'])

# or, to just see the first few values:
print(df['measurement_time'].head())


0        2017-03-31 23:59:56
1        2017-03-31 23:59:56
2        2017-03-31 23:59:56
3        2017-03-31 23:59:56
4        2017-03-31 23:59:56
                 ...        
966510   2017-03-19 15:41:15
966511   2017-03-19 15:40:12
966512   2017-03-19 15:39:10
966513   2017-03-19 15:38:08
966514   2017-03-19 15:37:07
Name: measurement_time, Length: 966515, dtype: datetime64[ns]
0   2017-03-31 23:59:56
1   2017-03-31 23:59:56
2   2017-03-31 23:59:56
3   2017-03-31 23:59:56
4   2017-03-31 23:59:56
Name: measurement_time, dtype: datetime64[ns]


In [11]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime

df_clean = df_combined.copy()



# Sensor
def correct_temperature_values(df):
    """
    Correct temperature values for sensors that report in mV.
    - For sensors with "MK-III" in the title and units indicating mV, divide the value by 10.
    - For sensors with "TM1" in the title and units indicating mV, convert using (mV - 400)/19.5.
    After conversion, update the units to 'Celsius'.
    """
    df_corrected = df.copy()
    if not isinstance(df_corrected.index, pd.RangeIndex):
         df_corrected = df_corrected.reset_index() # Ensure default index for loc

    # Ensure 'measurement_value' is numeric, coerce errors to NaN
    df_corrected['measurement_value'] = pd.to_numeric(df_corrected['measurement_value'], errors='coerce')

    # Correct MK-III sensors
    mask_mkiii = df_corrected['measurement_title'].astype(str).str.contains("MK-III", na=False)
    mask_mv_mkiii = mask_mkiii & df_corrected['units'].astype(str).str.lower().str.contains('mv', na=False)
    df_corrected.loc[mask_mv_mkiii, 'measurement_value'] = (
        df_corrected.loc[mask_mv_mkiii, 'measurement_value'] / 10.0 # Use float division
    )

    # Correct TM1 sensors if units indicate millivolts
    mask_tm1 = df_corrected['measurement_title'].astype(str).str.contains("TM1", na=False)
    mask_mv_tm1 = mask_tm1 & df_corrected['units'].astype(str).str.lower().str.contains('mv', na=False)
    df_corrected.loc[mask_mv_tm1, 'measurement_value'] = (
        (df_corrected.loc[mask_mv_tm1, 'measurement_value'] - 400.0) / 19.5 # Use float
    )

    # Update the units for corrected values
    df_corrected.loc[mask_mv_mkiii | mask_mv_tm1, 'units'] = 'Celsius'

    return df_corrected

df_clean = correct_temperature_values(df_clean)


df_clean['measurement_time'] = pd.to_datetime(df_clean['measurement_time'])

temp_data = df_clean[df_clean['measurement_type'] == 'Temperature'].copy()
humid_data = df_clean[df_clean['measurement_type'] == 'RelativeHumidity'].copy()
precip_data = df_clean[df_clean['measurement_type'] == 'CumulativePrecipitation'].copy()

# Group by day - Use dt.normalize() for consistency if time part matters initially
temp_data['day'] = temp_data['measurement_time'].dt.normalize()
humid_data['day'] = humid_data['measurement_time'].dt.normalize()
precip_data['day'] = precip_data['measurement_time'].dt.normalize()

# Aggregate daily values - handle potential non-numeric errors during mean calculation
daily_temp = temp_data.groupby('day')['measurement_value'].mean().reset_index()
daily_humid = humid_data.groupby('day')['measurement_value'].mean().reset_index()

precip_data['measurement_value'] = pd.to_numeric(precip_data['measurement_value'], errors='coerce')
precip_data.dropna(subset=['measurement_value'], inplace=True) # Drop rows where conversion failed

sensor_daily_precip = precip_data.groupby(['data_stream_id', 'day'])['measurement_value'].max().reset_index()
# Sort before diff to ensure correct calculation
sensor_daily_precip = sensor_daily_precip.sort_values(by=['data_stream_id', 'day'])
sensor_daily_precip['daily_change'] = sensor_daily_precip.groupby('data_stream_id')['measurement_value'].diff().fillna(0)
# Handle resets: Set negative diff values to 0
sensor_daily_precip.loc[sensor_daily_precip['daily_change'] < 0, 'daily_change'] = 0

# Aggregate daily change across sensors
daily_precip = sensor_daily_precip.groupby('day')['daily_change'].sum().reset_index()
# Convert mm to inches
daily_precip['daily_change'] = daily_precip['daily_change'] / 25.4
daily_precip.loc[daily_precip['daily_change'] > 3, 'daily_change'] = np.nan

# Merge datasets

daily_temp['day'] = pd.to_datetime(daily_temp['day'])
daily_humid['day'] = pd.to_datetime(daily_humid['day'])
daily_precip['day'] = pd.to_datetime(daily_precip['day'])

# Merge the daily aggregated data
daily_env = pd.merge(daily_temp, daily_humid, on='day', how='outer', suffixes=('_temp', '_humid'))
daily_env = pd.merge(daily_env, daily_precip[['day', 'daily_change']], on='day', how='left') # Left merge to keep all temp/humid days
# Rename columns for clarity
daily_env.rename(columns={
    'measurement_value_temp': 'temperature',
    'measurement_value_humid': 'humidity',
    'daily_change': 'precipitation'
}, inplace=True)

daily_env['day'] = pd.to_datetime(daily_env['day'])
daily_env_march = daily_env[
    (daily_env['day'] >= '2017-03-01') & (daily_env['day'] <= '2017-03-31')
].copy()

daily_env_march['precipitation'].fillna(0, inplace=True)


import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime


alt.data_transformers.disable_max_rows()

color_domain = ['Temperature', 'Humidity', 'Precipitation']
color_range = ['red', 'blue', 'darkseagreen']
color_scale = alt.Scale(domain=color_domain, range=color_range)

legend = alt.Legend(
    orient='bottom',
    title=None,
    direction='horizontal',
    labelFontSize=11,
    symbolStrokeWidth=2,
    padding=10
)


daily_env_until22 = daily_env[
    (daily_env['day'] >= pd.to_datetime('2017-03-01')) &
    (daily_env['day'] <= pd.to_datetime('2017-03-22'))
].copy()


base = alt.Chart(daily_env_until22).encode(
    x=alt.X('day:T',
            title='Date',
            axis=alt.Axis(format='%a %d', labelAngle=0, grid=True)),
)


temp_line = base.mark_line(point=True, strokeWidth=2).encode(
    y=alt.Y('temperature:Q',
            title='Temperature (°C)',
            axis=alt.Axis(titleColor='red', titlePadding=10, grid=True, gridColor='lightgray')),
    color=alt.Color('metric:N', scale=color_scale, legend=legend),
    tooltip=[alt.Tooltip('day:T', format='%Y-%m-%d (%a)', title='Date'),
             alt.Tooltip('temperature:Q', format='.1f', title='Temp (°C)')],
).transform_calculate(metric='"Temperature"')

humid_line = base.mark_line(point=True, strokeWidth=2).encode(
    y=alt.Y('humidity:Q',
            title='Relative Humidity (%)',
            axis=alt.Axis(orient='right', titleColor='blue', titlePadding=10, grid=False)),
    color=alt.Color('metric:N', scale=color_scale, legend=None),
    tooltip=[alt.Tooltip('day:T', format='%Y-%m-%d (%a)', title='Date'),
             alt.Tooltip('humidity:Q', format='.0f', title='Humidity (%)')],
).transform_calculate(metric='"Humidity"')

precip_bars = base.mark_bar(opacity=0.5, width=4).encode(
    y=alt.Y('precipitation:Q', axis=None),
    color=alt.Color('metric:N', scale=color_scale, legend=None),
    tooltip=[alt.Tooltip('day:T', format='%Y-%m-%d (%a)', title='Date'),
             alt.Tooltip('precipitation:Q', format='.2f', title='Precip (in)')],
).transform_calculate(metric='"Precipitation"')

precip_text = base.mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    fontSize=9,
    color='black'
).encode(
    y=alt.Y('precipitation:Q', axis=None),
    text=alt.Text('precipitation:Q', format='.2f'),
    opacity=alt.condition(alt.datum.precipitation > 0.001, alt.value(1), alt.value(0))
)


chart = alt.layer(
    precip_bars,
    temp_line,
    humid_line,
    precip_text
).resolve_scale(
    y='independent',
    color='shared'
).properties(
    width=700,
    height=400,
    title='Temperature, Humidity, and Precipitation in Chicago\n(Mar 1–22, 2017)'
).configure_axis(
    labelFontSize=10,
    titleFontSize=12,
    gridColor='lightgray'
).configure_title(
    fontSize=14,
    anchor='middle'
).configure_legend(
    padding=10,
    labelOffset=5
)

chart


import os, json



daily_env_until22.to_csv('data/daily_env_until22.csv', index=False)

# Save the Altair chart’s Vega-Lite spec:
with open('specs/env_spec_2.json', 'w') as f:
    json.dump(chart.to_dict(), f, indent=2)




/var/folders/cq/9yclshf57vx1b6p9t5xygfl80000gn/T/ipykernel_72743/649594001.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daily_env_march['precipitation'].fillna(0, inplace=True)
